In [1]:
import pandas as pd
from jsonpath_ng.ext import parse
import ujson
from tqdm import tqdm
import numpy as np
import cv2
import imagesize

tqdm.pandas()

In [2]:
df = pd.read_csv("rico/ui_details.csv")
df = df.drop(['Interaction Trace Number', 'UI Number in Trace', 'App Package Name'], axis=1)
df = df.rename(columns={"UI Number": "image_id"})
df

,image_id,name
0,0,com.brc.PeriodTrackerDiary
1,1,com.brc.PeriodTrackerDiary
2,2,yong.app.videoeditor
3,3,yong.app.videoeditor
4,4,yong.app.videoeditor
...,...,...
66256,72214,eu.mapof.mexico
66257,72215,eu.mapof.mexico
66258,72216,eu.mapof.mexico
66259,72217,eu.mapof.mexico


The semantic annotations are all 2k sized. We will scale them by the factor dependent on the resolution of the target image.

In [7]:
expr = parse("$..*[?(@.iconClass=='menu')].bounds")

def menu_vector(n, expr):
    with open(f'rico/semantic_annotations/{n}.json', 'r', encoding='utf-8') as f:
        data = ujson.load(f)
        width, height = imagesize.get(f'rico/combined/{n}.jpg')
        factor = (height/2560, width/1440)
    anchors = [[m.value[0]*factor[1], m.value[1]*factor[0], 
                m.value[2]*factor[1], m.value[3]*factor[0]] 
                for m in expr.find(data) 
                if (m.value[2]>m.value[0] and m.value[3]>m.value[1])]
    return anchors

df['anchors'] = df['image_id'].progress_apply(lambda n: menu_vector(n, expr))
df['image'] = df['image_id'].progress_apply(lambda n: f"{n}.jpg")
df

100%|██████████| 66261/66261 [00:00<00:00, 962892.77it/s]


,image_id,name,anchors,image
0,0,com.brc.PeriodTrackerDiary,[],0.jpg
1,1,com.brc.PeriodTrackerDiary,[],1.jpg
2,2,yong.app.videoeditor,[],2.jpg
3,3,yong.app.videoeditor,[],3.jpg
4,4,yong.app.videoeditor,[],4.jpg
...,...,...,...,...
66256,72214,eu.mapof.mexico,[],72214.jpg
66257,72215,eu.mapof.mexico,"[[352.5, 1198.5, 478.5, 1324.5]]",72215.jpg
66258,72216,eu.mapof.mexico,[],72216.jpg
66259,72217,eu.mapof.mexico,[],72217.jpg


In [8]:
def extract_anchors(anchor_values):
    if isinstance(anchor_values, list):
        return pd.Series([True]+anchor_values+[0, "burger"], index=['has_annotation', 'xmin', 'ymin', 'xmax', 'ymax', 'class_id', 'class_name'])
    return pd.Series([False]+[float("nan")]*5+["background"], index=['has_annotation', 'xmin', 'ymin', 'xmax', 'ymax', 'class_id', 'class_name'])

np.random.seed(42)
df = df.sample(frac=1).reset_index(drop=True)

split_index = int(len(df) * 0.8)
df['split'] = ['train' if i < split_index else 'val' for i in range(len(df))]
df = df.explode('anchors')
df[['has_annotation', 'xmin', 'ymin', 'xmax', 'ymax', 'class_id', 'class_name']] = df['anchors'].progress_apply(extract_anchors)
df = df.drop(columns=['anchors'], axis=1)
df

100%|██████████| 67486/67486 [00:13<00:00, 4975.13it/s]


,image_id,image,split,has_annotation,xmin,ymin,xmax,ymax,class_id,class_name
0,1943,1943.jpg,train,False,NaN,NaN,NaN,NaN,NaN,background
1,63707,63707.jpg,train,False,NaN,NaN,NaN,NaN,NaN,background
2,66859,66859.jpg,train,False,NaN,NaN,NaN,NaN,NaN,background
3,40815,40815.jpg,train,False,NaN,NaN,NaN,NaN,NaN,background
4,47210,47210.jpg,train,False,NaN,NaN,NaN,NaN,NaN,background
...,...,...,...,...,...,...,...,...,...,...
66256,40577,40577.jpg,val,False,NaN,NaN,NaN,NaN,NaN,background
66257,6653,6653.jpg,val,False,NaN,NaN,NaN,NaN,NaN,background
66258,59856,59856.jpg,val,False,NaN,NaN,NaN,NaN,NaN,background
66259,891,891.jpg,val,False,NaN,NaN,NaN,NaN,NaN,background


In [10]:
false_df = df[df['has_annotation'] == False]
true_df = df[df['has_annotation'] == True]

num_false = int(true_df['has_annotation'].count() * 0.2)
result_df = pd.concat([true_df, false_df.sample(n=num_false, random_state=42)])
result_df.to_csv("rico/burger.csv", index=False)
result_df

,image_id,image,split,has_annotation,xmin,ymin,xmax,ymax,class_id,class_name
6,12321,12321.jpg,train,True,0.0,63.0,147.00,210.00,0.0,burger
8,65445,65445.jpg,train,True,10.5,73.5,136.50,199.50,0.0,burger
16,56899,56899.jpg,train,True,0.0,63.0,131.25,194.25,0.0,burger
20,4850,4850.jpg,train,True,0.0,52.5,21.00,73.50,0.0,burger
23,56149,56149.jpg,train,True,0.0,63.0,147.00,210.00,0.0,burger
...,...,...,...,...,...,...,...,...,...,...
5747,18937,18937.jpg,train,False,NaN,NaN,NaN,NaN,NaN,background
49817,14823,14823.jpg,train,False,NaN,NaN,NaN,NaN,NaN,background
62774,32978,32978.jpg,val,False,NaN,NaN,NaN,NaN,NaN,background
1567,58932,58932.jpg,train,False,NaN,NaN,NaN,NaN,NaN,background
